# Ensemble by ranked list merging

In [1]:
%load_ext autoreload

from os import chdir
chdir('../')

from src.utils.loader import *
from src.utils.evaluator import Evaluator

dataset = Dataset(load_tags=True,
                  filter_tag=False,
                  weight_tag=False)
print("Dataset loaded...")

Dataset loaded...


In [2]:
from src.CBF.CBF_tfidf import ContentBasedFiltering
from src.UBF.UBF2 import UserBasedFiltering
from src.IBF.IBF import ItemBasedFiltering
from src.Pop.popularity import Popularity
from src.MF.MF_BPR.MF_BPR_CBF import MF_BPR_CBF
from src.MF.MF_BPR.MF_BPR_KNN import MF_BPR_KNN
from src.CBF.CBF_MF import ContentBasedFiltering as CBF_AUG
from src.FWUM.UICF import xSquared
from src.FWUM.UICF2 import UserItemFiltering

print("Models imported...")

Models imported...


## Initialize models
### SLIM BPR

In [1]:
%autoreload 2

from src.ML.BPRSLIM import BPRSLIM
from src.utils.matrix_utils import applyTfIdf, top_k_filtering

import os
dataset.set_track_attr_weights_2(1, 1, 0, 0, 0, 0, 0, 0, 0)
ev = Evaluator(seed=False)
ev.cross_validation(5, dataset.train_final.copy())

print('Building the ICM for SLIMBPR...')
icm = dataset.build_icm()

icm_tag = dataset.build_tags_matrix()

tags = applyTfIdf(icm_tag)

# Before stacking tags with the rest of the ICM, we keep only
# the top K tags for each item. This way we try to reduce the
# natural noise added by such sparse features.
tags = top_k_filtering(tags.transpose(), topK=55).transpose()
tags.data = np.ones_like(tags.data)

# stack all
icm = vstack([icm, tags], format='csr')

UsageError: Line magic function `%autoreload` not found.


In [4]:
slim = BPRSLIM(epochs=30,
               epochMultiplier=1.0,
               sgd_mode='rmsprop',
               learning_rate=5e-02,
               topK=300,
               urmSamplingChances=1 / 5,
               icmSamplingChances=4 / 5)

Building the ICM for SLIMBPR...
Compiling in Cython...
Compiled module saved in subfolder: /src/ML/Cython
Compilation complete!


In [31]:
# Initiliaze Models
dataset.set_track_attr_weights_2(1.5, 1.6, 0.0, 0.0, 0.0,
                                 1.0, 0.0, 0.0, 0.0)

# Models
cbf = ContentBasedFiltering()
ibf = ItemBasedFiltering()
ubf = UserBasedFiltering()
#mf_bpr = MF_BPR_CBF(r_hat_aug)
#mf_bpr_knn = MF_BPR_KNN(r_hat_aug)
xbf = xSquared()
uicf = UserItemFiltering()
pop = Popularity()
models = [cbf, ubf, ibf, pop, xbf, uicf]

print("Models initialized...")

Models initialized...


### SLIM BPR Fit

In [6]:
ev = Evaluator(seed=False)
ev.cross_validation(5, dataset.train_final.copy())
# Validation
urm, tg_tracks, tg_playlist = ev.get_fold(dataset)

slim.set_evaluation_every(10, ev)
slim.fit(urm.tocsr(),
         icm.tocsr(),
         tg_playlist,
         tg_tracks,
         dataset)

slim_recs = slim.predict()

File found, retrieving urm from it.
Load from file takes 0.69 seconds
Running fit process.
Processed 500000 ( 29.85% ) in 49.78 seconds. Sample per second: 10045
Processed 1000000 ( 59.70% ) in 18.66 seconds. Sample per second: 14779
Processed 1500000 ( 89.54% ) in 16.10 seconds. Sample per second: 18050
Processed 1675142 ( 100.00% ) in 5.20 seconds. Sample per second: 18993
Return S matrix to python caller...
Epoch 0 of 30 complete in 1.79 minutes
Processed 500000 ( 29.85% ) in 11.69 seconds. Sample per second: 42764
Processed 1000000 ( 59.70% ) in 12.20 seconds. Sample per second: 43098
Processed 1500000 ( 89.54% ) in 12.42 seconds. Sample per second: 42351
Processed 1675142 ( 100.00% ) in 4.76 seconds. Sample per second: 42131
Return S matrix to python caller...
Epoch 1 of 30 complete in 1.08 minutes
Processed 500000 ( 29.85% ) in 13.70 seconds. Sample per second: 36505
Processed 1000000 ( 59.70% ) in 12.87 seconds. Sample per second: 38652
Processed 1500000 ( 89.54% ) in 13.25 seco

/media/leonardoarcari/Data/Home/Development/python_projects/recsys_challenge/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


MAP@5: 0.09923923697504357
Epoch 10 of 30 complete in 1.84 minutes
Processed 500000 ( 29.85% ) in 22.61 seconds. Sample per second: 22118
Processed 1000000 ( 59.70% ) in 14.49 seconds. Sample per second: 27403
Processed 1500000 ( 89.54% ) in 13.63 seconds. Sample per second: 30225
Processed 1675142 ( 100.00% ) in 5.34 seconds. Sample per second: 30827
Return S matrix to python caller...
Epoch 11 of 30 complete in 1.69 minutes
Processed 500000 ( 29.85% ) in 25.37 seconds. Sample per second: 19706
Processed 1000000 ( 59.70% ) in 13.21 seconds. Sample per second: 26168
Processed 1500000 ( 89.54% ) in 13.38 seconds. Sample per second: 29192
Processed 1675142 ( 100.00% ) in 4.98 seconds. Sample per second: 29923
Return S matrix to python caller...
Epoch 12 of 30 complete in 1.72 minutes
Processed 500000 ( 29.85% ) in 22.78 seconds. Sample per second: 21948
Processed 1000000 ( 59.70% ) in 13.88 seconds. Sample per second: 27868
Processed 1500000 ( 89.54% ) in 13.60 seconds. Sample per second

In [7]:
# If not present, save SLIM BPR parameters to disk
w_path = './data/slim_bpr_parameters.npz'
if not os.path.isfile(w_path):
    save_sparse_matrix(w_path, slim.getParameters())

In [29]:
%load_ext autoreload
%autoreload 2
from src.Ensemble.ranked_list_merging import interleaved_merging, borda_count_ensemble
def merge_ranked_lists(recs):
    merged_recs = {}
    for playlist in recs[0].keys():
        lists = [rec[playlist] for rec in recs]
        # Pick 3 from CBF and 2 from IBF
        merged = interleaved_merging(lists, [3, 2], topK=5, mode="continue")
        merged_recs[playlist] = merged
    return merged_recs

def merge_borda(params):
    global recs, ev
    print(params)
    merged_recs = {}
    for playlist in recs[0].keys():
        lists = [rec[playlist] for rec in recs]
        #params = np.random.rand(len(recs))
        merged = borda_count_ensemble(lists, params, at=5)
        merged_recs[playlist] = merged
    map_at_five = ev.evaluate_fold(merged_recs)
    return -map_at_five

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
ev = Evaluator(seed=False)
ev.cross_validation(5, dataset.train_final.copy())
urm, tg_tracks, tg_playlist = ev.get_fold(dataset)
recs = []
for model in models:
    model.fit(urm,
             list(tg_playlist),
             list(tg_tracks),
             dataset)
    
    # append the current list to the 
    recs.append(model.predict(at=100))


CBF started
Build tags matrix and apply TFIDF...
Running 4 workers...
[ 11540 ] Building cosine similarity matrix for [0, 1000)...
[ 11541 ] Building cosine similarity matrix for [4336, 5336)...
[ 11542 ] Building cosine similarity matrix for [8672, 9672)...
[ 11543 ] Building cosine similarity matrix for [13008, 14008)...
[ 11540 ] Building cosine similarity matrix for [1000, 2000)...
[ 11541 ] Building cosine similarity matrix for [5336, 6336)...
[ 11542 ] Building cosine similarity matrix for [9672, 10672)...
[ 11543 ] Building cosine similarity matrix for [14008, 15008)...
[ 11540 ] Building cosine similarity matrix for [2000, 3000)...
[ 11541 ] Building cosine similarity matrix for [6336, 7336)...
[ 11542 ] Building cosine similarity matrix for [10672, 11672)...
[ 11543 ] Building cosine similarity matrix for [15008, 16008)...
[ 11540 ] Building cosine similarity matrix for [3000, 4000)...
[ 11541 ] Building cosine similarity matrix for [7336, 8336)...
[ 11542 ] Building cosine si

[ 11577 ] Building cosine similarity matrix for [9672, 9772)...
[ 11575 ] Building cosine similarity matrix for [2100, 2200)...
[ 11578 ] Building cosine similarity matrix for [13608, 13708)...
[ 11576 ] Building cosine similarity matrix for [6036, 6136)...
[ 11577 ] Building cosine similarity matrix for [9772, 9872)...
[ 11575 ] Building cosine similarity matrix for [2200, 2300)...
[ 11577 ] Building cosine similarity matrix for [9872, 9972)...
[ 11578 ] Building cosine similarity matrix for [13708, 13808)...
[ 11576 ] Building cosine similarity matrix for [6136, 6236)...
[ 11575 ] Building cosine similarity matrix for [2300, 2400)...
[ 11578 ] Building cosine similarity matrix for [13808, 13908)...
[ 11577 ] Building cosine similarity matrix for [9972, 10072)...
[ 11576 ] Building cosine similarity matrix for [6236, 6336)...
[ 11575 ] Building cosine similarity matrix for [2400, 2500)...
[ 11578 ] Building cosine similarity matrix for [13908, 14008)...
[ 11577 ] Building cosine simil

/Users/emanueleghelfi/Development/RS/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


FWUM started!
Start filtering
UFM ready
UCM ready
SHAPE of ICM:  (94658, 100000)
UFM and ICM Done!
Running 4 workers...
[ 11585 ] Building cosine similarity matrix for [0, 100)...
[ 11586 ] Building cosine similarity matrix for [1105, 1205)...
[ 11585 ] Building cosine similarity matrix for [100, 200)...
[ 11585 ] Building cosine similarity matrix for [200, 300)...
[ 11585 ] Building cosine similarity matrix for [300, 400)...
[ 11585 ] Building cosine similarity matrix for [400, 500)...
[ 11585 ] Building cosine similarity matrix for [500, 600)...
[ 11587 ] Building cosine similarity matrix for [2210, 2310)...
[ 11586 ] Building cosine similarity matrix for [1205, 1305)...
[ 11585 ] Building cosine similarity matrix for [600, 700)...
[ 11586 ] Building cosine similarity matrix for [1305, 1405)...
[ 11585 ] Building cosine similarity matrix for [700, 800)...
[ 11588 ] Building cosine similarity matrix for [3315, 3415)...
[ 11586 ] Building cosine similarity matrix for [1405, 1505)...
[ 

[ 11615 ] Building cosine similarity matrix for [6136, 6236)...
[ 11614 ] Building cosine similarity matrix for [2200, 2300)...
[ 11617 ] Building cosine similarity matrix for [14308, 14408)...
[ 11616 ] Building cosine similarity matrix for [10272, 10372)...
[ 11615 ] Building cosine similarity matrix for [6236, 6336)...
[ 11614 ] Building cosine similarity matrix for [2300, 2400)...
[ 11617 ] Building cosine similarity matrix for [14408, 14508)...
[ 11616 ] Building cosine similarity matrix for [10372, 10472)...
[ 11615 ] Building cosine similarity matrix for [6336, 6436)...
[ 11614 ] Building cosine similarity matrix for [2400, 2500)...
[ 11616 ] Building cosine similarity matrix for [10472, 10572)...
[ 11617 ] Building cosine similarity matrix for [14508, 14608)...
[ 11615 ] Building cosine similarity matrix for [6436, 6536)...
[ 11614 ] Building cosine similarity matrix for [2500, 2600)...
[ 11616 ] Building cosine similarity matrix for [10572, 10672)...
[ 11617 ] Building cosine 

In [35]:
from skopt import forest_minimize
from skopt.space import Integer, Real
space = [Real(0.0,1.0) for _ in range(len(models))]
x0 = [0 for _ in range(len(models))]
x0[0] = 1
res = forest_minimize(merge_borda, space, x0=x0, verbose=True, n_random_starts=30, n_calls=300, n_jobs=-1)
print('Maximimum p@k found: {:6.5f}'.format(-res.fun))
for p in res.x:
    print(p)

Iteration No: 1 started. Evaluating function at provided point.
[1, 0, 0, 0, 0, 0]
MAP@5: 0.11749151775616373
Iteration No: 1 ended. Evaluation done at provided point.
Time taken: 12.8890
Function value obtained: -0.1175
Current minimum: -0.1175
Iteration No: 2 started. Evaluating function at random point.
[0.65860484571642963, 0.28583885287545746, 0.99342956715481168, 0.10296629777585845, 0.009833374123833229, 0.78214888350865908]
MAP@5: 0.08877252506974279
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 14.5130
Function value obtained: -0.0888
Current minimum: -0.1175
Iteration No: 3 started. Evaluating function at random point.
[0.55527311609262786, 0.4594035773085563, 0.57787122182693196, 0.073559477082698846, 0.99468909252257354, 0.91907292587837053]
MAP@5: 0.09076754882002558
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 14.5301
Function value obtained: -0.0908
Current minimum: -0.1175
Iteration No: 4 started. Evaluating function at rando

MAP@5: 0.09604689738369895
Iteration No: 25 ended. Evaluation done at random point.
Time taken: 13.8662
Function value obtained: -0.0960
Current minimum: -0.1175
Iteration No: 26 started. Evaluating function at random point.
[0.56793110240963018, 0.46792624400002236, 0.96247315038069803, 0.20517618945503341, 0.89254699161834739, 0.99038832022712942]
MAP@5: 0.0906710397345998
Iteration No: 26 ended. Evaluation done at random point.
Time taken: 13.6333
Function value obtained: -0.0907
Current minimum: -0.1175
Iteration No: 27 started. Evaluating function at random point.
[0.045653417881037613, 0.4595150916775107, 0.4847482834420398, 0.32232550433447676, 0.92007348296412894, 0.95448007182866601]
MAP@5: 0.08779838648872793
Iteration No: 27 ended. Evaluation done at random point.
Time taken: 13.5046
Function value obtained: -0.0878
Current minimum: -0.1175
Iteration No: 28 started. Evaluating function at random point.
[0.13003299603792831, 0.40039188311961138, 0.50943688361849215, 0.2222765

MAP@5: 0.08468144462037268
Iteration No: 48 ended. Search finished for the next optimal point.
Time taken: 14.5286
Function value obtained: -0.0847
Current minimum: -0.1175
Iteration No: 49 started. Searching for the next optimal point.
[0.37243414534980396, 0.1210135956370654, 0.016704690169253026, 0.42188202776520578, 0.0080702067980613652, 0.020521705905927171]
MAP@5: 0.053628138430219054
Iteration No: 49 ended. Search finished for the next optimal point.
Time taken: 15.3485
Function value obtained: -0.0536
Current minimum: -0.1175
Iteration No: 50 started. Searching for the next optimal point.
[0.081278790585107355, 0.099086601807337221, 0.018321325855549245, 0.19553966505348369, 0.077987942087026876, 0.15254138823629884]
MAP@5: 0.05856819724044293
Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 15.3413
Function value obtained: -0.0586
Current minimum: -0.1175
Iteration No: 51 started. Searching for the next optimal point.
[0.10180704228043216, 0.007

MAP@5: 0.07302118676015973
Iteration No: 71 ended. Search finished for the next optimal point.
Time taken: 15.7181
Function value obtained: -0.0730
Current minimum: -0.1175
Iteration No: 72 started. Searching for the next optimal point.
[0.058590605520245323, 0.061437400278391854, 0.42670484653205931, 0.45631894855099087, 0.03167993570925632, 0.98475886204399143]
MAP@5: 0.0830204327829299
Iteration No: 72 ended. Search finished for the next optimal point.
Time taken: 14.5825
Function value obtained: -0.0830
Current minimum: -0.1175
Iteration No: 73 started. Searching for the next optimal point.
[0.36755875087262685, 0.032216728214082065, 0.37199568827897794, 0.8670458370106815, 0.19645296050419142, 0.014971522290406217]
MAP@5: 0.044550252582371754
Iteration No: 73 ended. Search finished for the next optimal point.
Time taken: 17.7943
Function value obtained: -0.0446
Current minimum: -0.1175
Iteration No: 74 started. Searching for the next optimal point.
[0.099670981545607412, 0.4637376

MAP@5: 0.07675714393425315
Iteration No: 94 ended. Search finished for the next optimal point.
Time taken: 15.6083
Function value obtained: -0.0768
Current minimum: -0.1175
Iteration No: 95 started. Searching for the next optimal point.
[0.42810922696843767, 0.010843264712800173, 0.39366366206538161, 0.70859587349763486, 0.0069440204008149892, 0.8050941661081753]
MAP@5: 0.07746286662142782
Iteration No: 95 ended. Search finished for the next optimal point.
Time taken: 15.6654
Function value obtained: -0.0775
Current minimum: -0.1175
Iteration No: 96 started. Searching for the next optimal point.
[0.5535581617300831, 0.0024377731493288039, 0.12439630158661977, 0.9525284840392354, 0.71720197437342648, 0.084617427952596316]
MAP@5: 0.0741498906733016
Iteration No: 96 ended. Search finished for the next optimal point.
Time taken: 16.2496
Function value obtained: -0.0741
Current minimum: -0.1175
Iteration No: 97 started. Searching for the next optimal point.
[0.13632701220915525, 0.100379346

...

MAP@5: 0.04976475910427472
Iteration No: 186 ended. Search finished for the next optimal point.
Time taken: 19.1995
Function value obtained: -0.0498
Current minimum: -0.1175
Iteration No: 187 started. Searching for the next optimal point.
[0.25070629205090483, 0.039031391240393128, 0.14154085160973998, 0.32059136988605003, 0.070901599239234239, 0.61529052277176666]
MAP@5: 0.08872502450425988
Iteration No: 187 ended. Search finished for the next optimal point.
Time taken: 18.1706
Function value obtained: -0.0887
Current minimum: -0.1175
Iteration No: 188 started. Searching for the next optimal point.
[0.0085485657103234764, 0.093183034403369072, 0.47963509669169524, 0.42845146935347944, 0.0017770420683129733, 0.97503303241904493]
MAP@5: 0.08127271356405023
Iteration No: 188 ended. Search finished for the next optimal point.
Time taken: 14.8466
Function value obtained: -0.0813
Current minimum: -0.1175
Iteration No: 189 started. Searching for the next optimal point.
[0.024113397328115578,

MAP@5: 0.07228304305209962
Iteration No: 209 ended. Search finished for the next optimal point.
Time taken: 15.4815
Function value obtained: -0.0723
Current minimum: -0.1175
Iteration No: 210 started. Searching for the next optimal point.
[0.13584200368126911, 0.3098439261510591, 0.006935720030454064, 0.56863243050500067, 0.35299892373986003, 0.23760398153467416]
MAP@5: 0.047665686496267526
Iteration No: 210 ended. Search finished for the next optimal point.
Time taken: 15.1257
Function value obtained: -0.0477
Current minimum: -0.1175
Iteration No: 211 started. Searching for the next optimal point.
[0.10310890778387208, 0.022583931971750352, 0.14344538760790551, 0.39377167711503802, 0.00086316230246796821, 0.41337529903797421]
MAP@5: 0.011761290809017595
Iteration No: 211 ended. Search finished for the next optimal point.
Time taken: 18.2814
Function value obtained: -0.0118
Current minimum: -0.1175
Iteration No: 212 started. Searching for the next optimal point.
[0.53615293630735861, 0

MAP@5: 0.08058207042147318
Iteration No: 232 ended. Search finished for the next optimal point.
Time taken: 14.6539
Function value obtained: -0.0806
Current minimum: -0.1175
Iteration No: 233 started. Searching for the next optimal point.
[0.071295991760324209, 0.050244556643947128, 0.14823092349360048, 0.29454413515655015, 0.19431601181388675, 0.36028498019899086]
MAP@5: 0.08171982206137356
Iteration No: 233 ended. Search finished for the next optimal point.
Time taken: 15.0509
Function value obtained: -0.0817
Current minimum: -0.1175
Iteration No: 234 started. Searching for the next optimal point.
[0.11102606767147108, 0.054073894599328334, 0.1758343053514714, 0.41179483027837482, 0.25940932213750595, 0.21244348909123673]
MAP@5: 0.0735180577546555
Iteration No: 234 ended. Search finished for the next optimal point.
Time taken: 15.3480
Function value obtained: -0.0735
Current minimum: -0.1175
Iteration No: 235 started. Searching for the next optimal point.
[0.10556096189463263, 0.0213

MAP@5: 0.08083465279348565
Iteration No: 255 ended. Search finished for the next optimal point.
Time taken: 14.3642
Function value obtained: -0.0808
Current minimum: -0.1175
Iteration No: 256 started. Searching for the next optimal point.
[0.7552091006344871, 0.05079483670577091, 0.015573229031363271, 0.80570264353128462, 0.80509402053670087, 0.81170609818594996]
MAP@5: 0.09224685214506523
Iteration No: 256 ended. Search finished for the next optimal point.
Time taken: 13.9286
Function value obtained: -0.0922
Current minimum: -0.1175
Iteration No: 257 started. Searching for the next optimal point.
[0.14512479937965164, 0.019491509757271545, 0.12046569155055212, 0.31333383860844327, 0.14909787742309824, 0.65864164386829849]
MAP@5: 0.08417250999019806
Iteration No: 257 ended. Search finished for the next optimal point.
Time taken: 14.5248
Function value obtained: -0.0842
Current minimum: -0.1175
Iteration No: 258 started. Searching for the next optimal point.
[0.65736518945503264, 0.0604

MAP@5: 0.09386111739425472
Iteration No: 278 ended. Search finished for the next optimal point.
Time taken: 13.8752
Function value obtained: -0.0939
Current minimum: -0.1175
Iteration No: 279 started. Searching for the next optimal point.
[0.2462847457523461, 0.018942379884814401, 0.026915882151454025, 0.2955433725355337, 0.069195604196926733, 0.28419564193471697]
MAP@5: 0.07404659579280683
Iteration No: 279 ended. Search finished for the next optimal point.
Time taken: 14.7407
Function value obtained: -0.0740
Current minimum: -0.1175
Iteration No: 280 started. Searching for the next optimal point.
[0.23102171075994066, 0.14291860751472976, 0.013410810305470092, 0.31274568281422338, 0.0040366909529230286, 0.20225740471415327]
MAP@5: 0.05292543165196378
Iteration No: 280 ended. Search finished for the next optimal point.
Time taken: 14.8950
Function value obtained: -0.0529
Current minimum: -0.1175
Iteration No: 281 started. Searching for the next optimal point.
[0.51904738422422647, 0.0

In [10]:
from src.Ensemble.ranked_list_merging import interleaved_merging, borda_count_ensemble



# Fit models
# CBF
cbf.fit(urm,
        list(tg_playlist),
        list(tg_tracks),
        dataset)
cbf_recs = cbf.predict()
print("Evaluating CBF...")
ev.evaluate_fold(cbf_recs)

# IBF
ibf.fit(urm,
        list(tg_playlist),
        list(tg_tracks),
        dataset)
ibf_recs = ibf.predict()
print("Evaluating IBF...")
ev.evaluate_fold(ibf_recs)

print("Evaluating SLIM BPR...")
ev.evaluate_fold(slim_recs)

merged_recs = merge_ranked_lists([cbf_recs, slim_recs])
print("Evaluating Merged recs...")
ev.evaluate_fold(merged_recs)

CBF started
Build tags matrix and apply TFIDF...
Running 4 workers...
[ 1303 ] Building cosine similarity matrix for [0, 1000)...
[ 1304 ] Building cosine similarity matrix for [4319, 5319)...
[ 1305 ] Building cosine similarity matrix for [8638, 9638)...
[ 1306 ] Building cosine similarity matrix for [12957, 13957)...
[ 1305 ] Building cosine similarity matrix for [9638, 10638)...
[ 1303 ] Building cosine similarity matrix for [1000, 2000)...
[ 1304 ] Building cosine similarity matrix for [5319, 6319)...
[ 1306 ] Building cosine similarity matrix for [13957, 14957)...
[ 1305 ] Building cosine similarity matrix for [10638, 11638)...
[ 1304 ] Building cosine similarity matrix for [6319, 7319)...
[ 1306 ] Building cosine similarity matrix for [14957, 15957)...
[ 1303 ] Building cosine similarity matrix for [2000, 3000)...
[ 1305 ] Building cosine similarity matrix for [11638, 12638)...
[ 1306 ] Building cosine similarity matrix for [15957, 16957)...
[ 1303 ] Building cosine similarity ma

0.11588253034758354